In [1]:
!pip install farm-haystack[preprocessing] -q
!pip install farm-haystack[inference] -q
!pip install nltk -q

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_visitable_links(base_url):
    # Send a GET request to the base URL
    response = requests.get(base_url)
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve {base_url}")
        return []
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    
    # Find all anchor tags in the page
    for a_tag in soup.find_all('a', href=True):
        # Get the full URL
        full_url = urljoin(base_url, a_tag['href'])
        
        # Check if the URL is visitable (not a mailto or javascript link)
        if full_url.startswith(base_url):
            links.append(full_url)
    
    return links

# Example usage
base_url = 'https://github.com/hissain/'  # Replace with your desired URL
visitable_links = get_visitable_links(base_url)

# Print the collected links
for link in visitable_links:
    print(link)


https://github.com/hissain/#start-of-content
https://github.com/hissain/
https://github.com/hissain/
https://github.com/hissain/
https://github.com/hissain/hissain
https://github.com/hissain/#about-me
https://github.com/hissain/#stackoverflow
https://github.com/hissain/CoronaTracker
https://github.com/hissain/CoronaTracker/stargazers
https://github.com/hissain/CoronaTracker/forks
https://github.com/hissain/minimal-cmake
https://github.com/hissain/minimal-cmsis-dsp
https://github.com/hissain/mlworks
https://github.com/hissain/mlworks/stargazers
https://github.com/hissain/dsp-spectrogram


In [5]:
# Step 1: Scrape webpages and prepare documents

import requests
from bs4 import BeautifulSoup
from haystack.schema import Document

# Replace this list with your actual list of blog URLs
urls = [
    'https://en.wikipedia.org/wiki/Bangladesh',
    'https://en.wikipedia.org/wiki/India',
    # Add more URLs as needed
]

urls = visitable_links

documents = []

for url in urls:
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text from all paragraph tags
        paragraphs = soup.find_all('p')
        content = '\n'.join([para.get_text() for para in paragraphs])

        # Create a Haystack Document with content and optional metadata
        doc = Document(content=content, meta={'url': url})
        documents.append(doc)
    except Exception as e:
        print(f'Error processing {url}: {e}')

In [6]:
len(documents)

15

In [9]:
from haystack.nodes import PreProcessor
from haystack.schema import Document

# Initialize PreProcessor with desired settings
preprocessor = PreProcessor(
    split_by="word",            # Can also be "sentence" if sentence-based splitting is preferred
    split_length=300,           # Number of words per chunk
    split_respect_sentence_boundary=True,  # Ensure splitting occurs at sentence boundaries
    split_overlap=50            # Overlap between chunks to preserve context
)

# Process the scraped documents to split them
processed_documents = preprocessor.process(documents)

# Now, 'processed_documents' will contain smaller chunks ready for the document store
document_store.write_documents(processed_documents)

# Verify the number of chunks after splitting
print(f"Number of documents after splitting: {document_store.get_document_count()}")

# Example: Print a chunk to inspect
sample_chunk = document_store.get_all_documents()[0]
print(sample_chunk.content)
print(sample_chunk.meta)


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
dlopen(/Users/hissain/anaconda3/lib/python3.11/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so, 2): Library not loaded: @rpath/libtensorflow_cc.2.dylib
  Referenced from: /Users/hissain/anaconda3/lib/python3.11/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so
  Reason: image not found

In [10]:
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import DensePassageRetriever, FARMReader, BM25Retriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
dlopen(/Users/hissain/anaconda3/lib/python3.11/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so, 2): Library not loaded: @rpath/libtensorflow_cc.2.dylib
  Referenced from: /Users/hissain/anaconda3/lib/python3.11/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so
  Reason: image not found